# Importing Libraries

In [13]:
import numpy as np
import pandas as pd
from numpy import mean
from numpy import absolute
from numpy import sqrt

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score,mean_squared_error,r2_score

# Importing Datasets

In [4]:
train=pd.read_csv("train_dataset.csv")

In [5]:
test=pd.read_csv("test_dataset.csv")

# Extracting Data

In [6]:
train=train.set_index('Unnamed: 0')
X_train=train.iloc[:,0:9]
y_train=train.Survived

In [7]:
test=test.set_index('Unnamed: 0')
X_test=test.iloc[:,0:9]
y_test=test.Survived

# Taking a look at the data

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(534, 9)
(534,)
(179, 9)
(179,)


In [9]:
X_train.head()

,Pclass,Sex,Age,Fare,Embarked,FamilyCount,Title,Has Cabin,IsAlone
Unnamed: 0,,,,,,,,,
711,1,0,29.699118,26.5500,0,0,1,1,0
466,2,0,29.699118,0.0000,0,0,1,0,0
213,2,0,30.000000,13.0000,0,0,1,0,0
496,1,1,54.000000,78.2667,2,1,2,1,1
583,1,0,36.000000,40.1250,2,0,1,1,0


In [10]:
y_train.head()

Unnamed: 0
711    0
466    0
213    0
496    1
583    0
Name: Survived, dtype: int64

# RandomForest Classifier Model

In [33]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred=rf.predict(X_test)
acc_rf= accuracy_score(y_test,y_pred)
acc_rf

0.8044692737430168

# Cross validating the model

In [34]:
#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [35]:
scores = cross_val_score(rf, X_train, y_train, scoring='neg_mean_absolute_error',cv=cv, n_jobs=-1)

#view mean absolute error
mean(absolute(scores))

0.1758211041229909

In [36]:
#define cross-validation method to use
cv = KFold(n_splits=5, random_state=1, shuffle=True) 

#build random forest claassifier model
rf = RandomForestClassifier()

#use LOOCV to evaluate model
scores = cross_val_score(rf, X_train, y_train, scoring='neg_mean_squared_error',cv=cv, n_jobs=-1)

#view RMSE
sqrt(mean(absolute(scores)))

0.43278612094847574

Both MAE and RMSE are quite low meaning our model's performane is quite good

# Pickle the model

In [31]:
# save the model to disc
import pickle
filename = "RandomForestClassifier model.pkl"
pickle.dump(rf,open(filename,'wb'))